In [10]:
import itertools
import os.path
from mltools.weka import *
from mltools.perfmeasures import * 
from mltools.mltools import * 

## run profkernel

In [15]:
l = 5
y = 15
fraglen = 25
method = "bval"
path = "data/testset{0}.{1}/testset{0}.{1}".format(fraglen, method)

In [16]:
print(">>> Working on parameters L={0}, Y={1}".format(l, y))
!profkernel-core -o {path}.idfile -K -L {l} -Y {y} -i /usr/share/fastprofkernel/data/Amino.txt -g /usr/share/fastprofkernel/data/Globals.txt {path}.kernelinput > {path}_L{l}_Y{y}.mat 2> {path}_L{l}_Y{y}.mat.log

>>> Working on parameters L=5, Y=15


In [17]:
row_count = !wc -l "{path}_L{l}_Y{y}.mat"
row_count = int(row_count[0].split(" ")[0])
row_count

2098

In [18]:
!echo "{row_count} {row_count}\n$(cat {path}_L{l}_Y{y}.mat)" > {path}_L{l}_Y{y}.2.mat

## run weka

In [11]:
!java -Xmx5g -cp bin/wekaTobi.jar:bin/weka.jar: weka.classifiers.meta.CVParameterSelection -p 1 -t {path}.train.arff -T {path}.test.arff -S 1 -W weka.classifiers.functions.SMO -- -C 1.0 -L 0.0010 -P 1.0E-12 -N 0 -V -1 -W 1 -K "weka.classifiers.functions.supportVector.CustomPrecomputedKernelMatrixKernelFast -M {path}_L{l}_Y{y}.2.mat" > {path}_L{l}_Y{y}.weka


In [12]:
perf_dict = {}
instances = extract_confusion_matrix_instances("{0}_L{1}_Y{2}.weka".format(path, l, y))
perfs = bootstrap(instances)

In [13]:
mccs = []


for perf in perfs: 
    mccs.append(mcc(*perf))

In [14]:
print(np.mean(mccs))
print(stderr(mccs))

0.135266712354
0.000445417215459
